In [ ]:
import numpy as np
import time as time
import os
import constants

# Set up matplotlib and use a nicer set of plot parameters
# %config InlineBackend.rc = {}
import matplotlib as mpl
# mpl.rc('mathtext',fontset='stixsans')
# mpl.rc('figure', facecolor="white")
#matplotlib.rc_file("../../templates/matplotlibrc")
import matplotlib.pyplot as plt
#import matplotlib.colors as colors
# %matplotlib notebook
import scipy.optimize

from astropy.cosmology import FlatLambdaCDM
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord

# Define cosmology
cosmo = constants.COSMOLOGY

PiBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins23_pi_0-30hMpc.txt')
SigBin_fil = os.path.join(constants.XCORR_DIR_BASE, 'bins10_sigma_0-30hMpc.txt')

PiBins0 = ascii.read(PiBin_fil)
SigBins0 = ascii.read(SigBin_fil)

PiEdges = PiBins0['pi_edges'].data
SigEdges = SigBins0['sigma_edges'].data

# Convert bin boundaries from Mpc/h to Mpc
#PiEdges  = PiEdges/(len(PiEdges)*[cosmo.h])
#SigEdges = SigEdges/(len(SigEdges)*[cosmo.h])

print('Pi bin edges in Mpc:')
print(PiEdges)
print('Sigma bin edges in Mpc:')
print(SigEdges)

PiBound = (min(PiEdges), max(PiEdges) )

In [ ]:
OBS_DIR = os.path.join(constants.XCORR_DIR_BASE, 'obs')

XCorr_3d=np.load(os.path.join(OBS_DIR, f"xcorr_3dhst_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_3d = np.transpose(XCorr_3d)
    
XCorr_zD = np.load(os.path.join(OBS_DIR, f"xcorr_zDeep_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_zD = np.transpose(XCorr_zD)
    
XCorr_mosdef = np.load(os.path.join(OBS_DIR, f"xcorr_mosdef_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_mosdef = np.transpose(XCorr_mosdef)
    
XCorr_vuds = np.load(os.path.join(OBS_DIR, f"xcorr_vuds_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_vuds = np.transpose(XCorr_vuds)

XCorr_clamato = np.load(os.path.join(OBS_DIR, f"xcorr_clamato_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_clamato = np.transpose(XCorr_clamato)

XCorr_all = np.load(os.path.join(OBS_DIR, f"xcorr_all_globalf_{constants.DATA_VERSION}.npy"))
XCorrPlot_all = np.transpose(XCorr_all)

print(XCorr_all.shape)

xcorr_list = [XCorr_3d, XCorr_zD, XCorr_mosdef, XCorr_vuds, XCorr_clamato]
titles = ['3DHST', 'zDeep', 'MOSDEF', 'VUDS', 'CLAMATO']

In [ ]:
COVAR_DIR = os.path.join(constants.XCORR_DIR_BASE, 'bootstrap')

Covar_zdeep = np.load(os.path.join(COVAR_DIR, f'covar_zDeep_n7200_{constants.DATA_VERSION}.npy'))
Covar_vuds  = np.load(os.path.join(COVAR_DIR, f'covar_VUDS_n7200_{constants.DATA_VERSION}.npy'))
Covar_clamato = np.load(os.path.join(COVAR_DIR, f'covar_CLAMATO_n7200_{constants.DATA_VERSION}.npy'))
Covar_mosdef = np.load(os.path.join(COVAR_DIR, f'covar_MOSDEF_n7200_{constants.DATA_VERSION}.npy'))
Covar_3dhst = np.load(os.path.join(COVAR_DIR, f'covar_3DHST_n7200_{constants.DATA_VERSION}.npy'))

covar_list = [Covar_3dhst, Covar_zdeep, Covar_mosdef, Covar_vuds, Covar_clamato]

In [ ]:
# Reduce covar in adding.
def reduce_covar(covar):
    parallel_size = XCorr_all.shape[1]
    if covar.ndim != 4:
        if covar.shape[0] == XCorr_all.size:
            covar = covar.reshape(*XCorr_all.shape, *XCorr_all.shape)
        else:
            raise RuntimeError
    return np.sum(covar, axis=(0, 2))
    
reduced_covar_list = []
for covar in covar_list:
    reduced_covar_list.append(reduce_covar(np.copy(covar)))
    assert np.allclose(reduced_covar_list[-1], reduced_covar_list[-1].T)

In [ ]:
def peak(x, *p):
    A, mu, C = p
    sigma = 23
    dist = np.minimum(np.abs(x - (mu + sigma)), np.abs(x - (mu - sigma)))
    dist[x > mu + sigma] = 0
    dist[x < mu - sigma] = 0
    return A * dist**3 + C
    #return A*np.exp(-(x-mu)**2/(2.*sigma**2)) + C

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(11, 6))

for xcorr, covar, t, ax in zip(xcorr_list, reduced_covar_list, titles, axes.flatten()):
    reduced_xcorr = np.sum(xcorr, axis=0)
    ax.errorbar(PiEdges[:-1], reduced_xcorr, yerr=np.diag(covar)**0.5, color='black')
    background = np.mean([reduced_xcorr[0], reduced_xcorr[-1]])
    fit_coeff, _ = scipy.optimize.curve_fit(peak, PiEdges[:-1], reduced_xcorr, p0=[(np.min(reduced_xcorr) - background) / 23**3, 0, background], sigma=covar)
    print(fit_coeff)
    ax.plot(PiEdges[:-1], peak(PiEdges[:-1], *fit_coeff), color='red')
    ax.set_title(f'{t} delta={-fit_coeff[1]:.2}')

In [ ]:
plt.imshow(reduced_covar_list[4])